In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd gdrive/My\ Drive/KKB-kaggle/bengaliai-cv19/notebooks

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/
/content/gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks


In [2]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=9b2380f1ebcacb788cf3f3cb507472dba7c252bcc82dbdc4073ff1a9c029bfb1
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built efficientnet-pytorch


In [0]:
## 諸々の import

import numpy as np
import pandas as pd
import cv2
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
import torchvision.models as models
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import PIL
import gc


from preprocess import *


import model.CNN as CNN
import model.efficientnet as efficientnet
import model.resnet18 as resnet18
import model.resnet34 as resnet34
import model.resnet50 as resnet50
import model.resnet101 as resnet101
import model.resnet152 as resnet152

In [0]:
## Parameters

# resize後のサイズ
HEIGHT = 64
WIDTH = 64

# 画像を3次元にするかどうか（EfficientNetなどを使うときはTrue）
# enable_3d = True
# ResNetとEfficientNetしか使わないので、常にTrueを想定する

# True なら submission.csv を生成する
create_submission = False

#Kaggleで提出するときはTrueにする
kaggle_flag = False

In [0]:
if kaggle_flag:
    dataset_dir = '/kaggle/input/bengaliai-cv19'
    model_dir = '/kaggle/input/trained_models'
else:
    dataset_dir = '../dataset'
    model_dir = '../trained_models'

train_df = pd.read_csv(dataset_dir + '/train.csv')
test_df = pd.read_csv(dataset_dir + '/test.csv')
class_map_df = pd.read_csv(dataset_dir + '/class_map.csv')
sample_sub_df = pd.read_csv(dataset_dir + '/sample_submission.csv')


In [21]:
model1 = resnet18.model()
model2 = efficientnet.model()


ensemble = [
    {
        'model': model1,
        'file': 'resnet18.pth',
        'w_grapheme': 0.30,
        'w_vowel': 0.30,
        'w_consonant': 0.30
    },
    {
        'model': model2,
        'file': 'efficientnet_28epoch.pth',
        'w_grapheme': 0.70,
        'w_vowel': 0.70,
        'w_consonant': 0.70
    }
]

Loaded pretrained weights for efficientnet-b0


In [0]:
# テスト用のDataset, DataLoaderを作りたい

class TestDataset(torch.utils.data.Dataset):

    def __init__(self, X, transform=None, H=64, W=64):
        self.transform = transform
        self.X = X
        self.H = H 
        self.W = W 

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):

        out_data = cv2.resize(self.X[idx].reshape(self.H, self.W), (224, 224), interpolation=cv2.INTER_AREA)
        out_data = out_data.reshape(224, 224, 1).astype(np.uint8)
        out_data = cv2.cvtColor(out_data, cv2.COLOR_GRAY2RGB)
        out_data = np.transpose(out_data, (2,0,1)) / 255.0
        out_data = out_data.reshape(3,224,224) 
        
        out_data = torch.tensor(out_data, dtype=torch.float)

        if self.transform:
            out_data = self.transform(out_data)

        return out_data

In [23]:
## 提出ファイルの作成

target=[]
row_id=[] # row_id place holder

for parq_i in range(4):
    df_test_img = pd.read_parquet(dataset_dir + f'/test_image_data_{parq_i}.parquet')
    # df_test_img = pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet') # Error Check!
    df_test_img.set_index('image_id', inplace=True)

    X_test_resized = resize(df_test_img, out_height=HEIGHT, out_width=WIDTH).astype(np.uint8)

    parq_size = len(df_test_img)
    pred_r = [None] * parq_size
    pred_v = [None] * parq_size
    pred_c = [None] * parq_size
    
    index = 0
    id_names = df_test_img.index.values

    test_dataset = TestDataset(X_test_resized, H=HEIGHT, W=WIDTH)
    test_loader = DataLoader(test_dataset, batch_size=64, num_workers=0)

    for data in test_loader:
        data = Variable(data)
        data = try_gpu(data)

        bs = len(data)
        root_ens, vowel_ens, consonant_ens = np.zeros((bs, 168)), np.zeros((bs, 11)), np.zeros((bs, 7))
        # ens = ensembleの意味を込めて

        for M in ensemble:
            device = torch.device('cpu')
            model = M['model']
            model.load_state_dict(torch.load(model_dir + '/' + M['file'], map_location=device))
            model = try_gpu(model)
            print("model loaded from {}".format(model_dir + '/' + M['file']))

            root_o, vowel_o, consonant_o = model(data)
            root_ens += M['w_grapheme'] * root_o.data.numpy()
            vowel_ens += M['w_vowel'] * vowel_o.data.numpy()
            consonant_ens += M['w_consonant'] * consonant_o.data.numpy()

        root_pred, vowel_pred, consonant_pred = np.argmax(root_ens, axis=1), np.argmax(vowel_ens, axis=1), np.argmax(consonant_ens, axis=1)

        for i in range(bs):
            row_id.append(id_names[index]+'_consonant_diacritic')
            target.append(consonant_pred[i])
            row_id.append(id_names[index]+'_grapheme_root')
            target.append(root_pred[i])
            row_id.append(id_names[index]+'_vowel_diacritic')
            target.append(vowel_pred[i])
            index += 1
    
    del df_test_img
    del X_test_resized
    del test_dataset
    del test_loader
    gc.collect()


df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)

if create_submission:
    df_sample.to_csv('submission.csv',index=False)

df_sample.head(36)

Resizing raw image... / 前処理実行中…
model loaded from ../trained_models/resnet18.pth
model loaded from ../trained_models/efficientnet_28epoch.pth
Resizing raw image... / 前処理実行中…
model loaded from ../trained_models/resnet18.pth
model loaded from ../trained_models/efficientnet_28epoch.pth
Resizing raw image... / 前処理実行中…
model loaded from ../trained_models/resnet18.pth
model loaded from ../trained_models/efficientnet_28epoch.pth
Resizing raw image... / 前処理実行中…
model loaded from ../trained_models/resnet18.pth
model loaded from ../trained_models/efficientnet_28epoch.pth


,row_id,target
0,Test_0_consonant_diacritic,0
1,Test_0_grapheme_root,3
2,Test_0_vowel_diacritic,0
3,Test_1_consonant_diacritic,0
4,Test_1_grapheme_root,93
5,Test_1_vowel_diacritic,2
6,Test_2_consonant_diacritic,0
7,Test_2_grapheme_root,19
8,Test_2_vowel_diacritic,0
9,Test_3_consonant_diacritic,0
